In [ ]:
import collections
import pickle
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from matrix_factorization import MF, NCF_ours, NCF
from utils import *
from sklearn.metrics import roc_auc_score
import time
np.random.seed(2020)
torch.manual_seed(2020)

def mse_func(x, y): return np.mean((x-y)**2)

file = open("constructed_data", "rb")
constructed_data_train = pickle.load(file)
constructed_data_test = pickle.load(file)
x_tr = pickle.load(file)
t_tr = pickle.load(file)
c_tr = pickle.load(file)
y_tr = pickle.load(file)
file.close()

num_user = int(max(x_tr[:, 0]) + 1)
num_item = int(max(x_tr[:, 1]) + 1)

In [ ]:
embedding_k = 64
C = 0.4
budget = 0.4
batch_size = 8192
scenario = 'coupon'
x_te = constructed_data_test[:, :2]
R = np.zeros(constructed_data_test.shape[0])
if scenario == 'cash':
    for i in range(constructed_data_test.shape[0]):
        if constructed_data_test[:, 6][i] == 4:
            R[i] = 1
        elif constructed_data_test[:, 6][i] == 3 or constructed_data_test[:, 6][i] == 5:
            R[i] = -C
if scenario == 'coupon':
    for i in range(constructed_data_test.shape[0]):
        if constructed_data_test[:, 6][i] == 4:
            R[i] = 1
        elif constructed_data_test[:, 6][i] == 5:
            R[i] = -C
            
constructed_data_test = np.c_[constructed_data_test, R]   

In [ ]:
ncf_y0or = NCF(num_user, num_item, embedding_k = embedding_k)
ncf_y0or.cuda()
ncf_y0or.fit(x_tr[t_tr == 0], y_tr[t_tr == 0], lamb = 1e-5, lr = 0.01, batch_size = 8192)
y0_or, _ = ncf_y0or.predict(x_tr)
y0_or_te, _ = ncf_y0or.predict(x_te)

ncf_y1or = NCF(num_user, num_item, embedding_k = embedding_k)
ncf_y1or.cuda()
ncf_y1or.fit(x_tr[t_tr == 1], y_tr[t_tr == 1], lamb = 1e-5, lr = 0.01, batch_size = 8192)
y1_or, _  = ncf_y1or.predict(x_tr)
y1_or_te, _  = ncf_y1or.predict(x_te)

reward_or = (1 + C) * (y1_or_te - y0_or_te) - C
temp_or = np.c_[constructed_data_test, reward_or]
temp_or = temp_or[np.argsort(-temp_or[:, -1])]

In [ ]:
reward_naive = y1_or_te - C
temp_naive = np.c_[constructed_data_test, reward_naive]
temp_naive = temp_naive[np.argsort(-temp_naive[:, -1])]

In [ ]:
ncf = NCF(num_user, num_item, embedding_k = embedding_k)
ncf.cuda()
ncf.fit(x_tr[t_tr == 1], c_tr[t_tr == 1], lamb = 1e-5, lr = 0.01, batch_size = 8192)
c1_te, _ = ncf.predict(x_te)
y1_or_c1_te = c1_te * y1_or_te
y1_or_c0_te = (1-c1_te) * y1_or_te
y0_or_c1_te = c1_te * (1-y1_or_te)
y0_or_c0_te = (1-c1_te) * (1-y1_or_te)

pa_or = y0_or_c0_te
pb_or = y1_or_c0_te
pc_or = y0_or_c1_te
pe_or = y0_or_te - y1_or_c0_te
pd_or = y1_or_c1_te - pe_or

if scenario == 'cash':
    cost = np.array([0, 0, -C, 1, -C])
    
if scenario == 'coupon':
    cost = np.array([0, 0, 0, 1, -C])    
    
p_or = np.c_[pa_or, pb_or, pc_or, pd_or, pe_or]    
    
reward_or = np.sum(p_or * cost, axis = 1)

temp_or_ours = np.c_[constructed_data_test, reward_or]
temp_or_ours = temp_or_ours[np.argsort(-temp_or_ours[:, -1])]

In [ ]:
temp_5 = temp_or_ours[:int(min(budget, sum(temp_or_ours[:, -1] > 0)/temp_or_ours.shape[0]) * temp_or_ours.shape[0])]
a = sum(temp_5[:, 6] == 1)
b = sum(temp_5[:, 6] == 2)
c = sum(temp_5[:, 6] == 3)
d = sum(temp_5[:, 6] == 4)
e = sum(temp_5[:, 6] == 5)
Real_reward = sum(temp_5[:, -2])
print(a, b, c, d, e, Real_reward)

In [ ]:
temp_5 = temp_or[:int(min(budget, sum(temp_or[:, -1] > 0)/temp_or.shape[0]) * temp_or.shape[0])]
a = sum(temp_5[:, 6] == 1)
b = sum(temp_5[:, 6] == 2)
c = sum(temp_5[:, 6] == 3)
d = sum(temp_5[:, 6] == 4)
e = sum(temp_5[:, 6] == 5)
Real_reward = sum(temp_5[:, -2])
print(a, b, c, d, e, Real_reward)

In [ ]:
temp_5 = temp_naive[:int(min(budget, sum(temp_naive[:, -1] > 0)/temp_naive.shape[0]) * temp_naive.shape[0])]
a = sum(temp_5[:, 6] == 1)
b = sum(temp_5[:, 6] == 2)
c = sum(temp_5[:, 6] == 3)
d = sum(temp_5[:, 6] == 4)
e = sum(temp_5[:, 6] == 5)
Real_reward = sum(temp_5[:, -2])
print(a, b, c, d, e, Real_reward)

In [ ]:
ncf_ps = NCF(num_user, num_item, embedding_k = embedding_k)
ncf_ps.cuda()
ncf_ps.fit(x_tr, t_tr, lamb = 1e-5, lr = 0.01, batch_size = 8192)
ps_tr, _ = ncf_ps.predict(x_tr)
ps_tr = np.clip(ps_tr, 0.1, 0.9)

In [ ]:
ncf_y0ips = NCF(num_user, num_item, embedding_k = embedding_k)
ncf_y0ips.cuda()
ncf_y0ips.fit(x_tr, (1-t_tr) * (y_tr/(1-ps_tr)), lamb = 1e-5, lr = 0.01, batch_size = 8192)
y0_ips, _  = ncf_y0ips.predict(x_tr)
y0_ips_te, _  = ncf_y0ips.predict(x_te)

ncf_y1ips = NCF(num_user, num_item, embedding_k = embedding_k)
ncf_y1ips.cuda()
ncf_y1ips.fit(x_tr, t_tr * y_tr/(ps_tr), lamb = 1e-5, lr = 0.01, batch_size = 8192)
y1_ips, _  = ncf_y1ips.predict(x_tr)
y1_ips_te, _  = ncf_y1ips.predict(x_te)

reward_ips = (1 + C) * (y1_ips_te - y0_ips_te) - C
temp_ips = np.c_[constructed_data_test, reward_ips]
temp_ips = temp_ips[np.argsort(-temp_ips[:, -1])]

In [ ]:
y1_ips_c1_te = c1_te * y1_ips_te
y1_ips_c0_te = (1-c1_te) * y1_ips_te
y0_ips_c1_te = c1_te * (1-y1_ips_te)
y0_ips_c0_te = (1-c1_te) * (1-y1_ips_te)

pa_ips = y0_ips_c0_te
pb_ips = y1_ips_c0_te
pc_ips = y0_ips_c1_te
pe_ips = y0_ips_te - y1_ips_c0_te
pd_ips = y1_ips_c1_te - pe_ips

if scenario == 'cash':
    cost = np.array([0, 0, -C, 1, -C])
    
if scenario == 'coupon':
    cost = np.array([0, 0, 0, 1, -C])    
    
p_ips = np.c_[pa_ips, pb_ips, pc_ips, pd_ips, pe_ips]    
    
reward_ips = np.sum(p_ips * cost, axis = 1)

temp_ips_ours = np.c_[constructed_data_test, reward_ips]
temp_ips_ours = temp_ips_ours[np.argsort(-temp_ips_ours[:, -1])]

In [ ]:
temp_5 = temp_ips_ours[:int(min(budget, sum(temp_ips_ours[:, -1] > 0)/temp_ips_ours.shape[0]) * temp_ips_ours.shape[0])]
a = sum(temp_5[:, 6] == 1)
b = sum(temp_5[:, 6] == 2)
c = sum(temp_5[:, 6] == 3)
d = sum(temp_5[:, 6] == 4)
e = sum(temp_5[:, 6] == 5)
Real_reward = sum(temp_5[:, -2])
print(a, b, c, d, e, Real_reward)

In [ ]:
temp_5 = temp_ips[:int(min(budget, sum(temp_ips[:, -1] > 0)/temp_ips.shape[0]) * temp_ips.shape[0])]
a = sum(temp_5[:, 6] == 1)
b = sum(temp_5[:, 6] == 2)
c = sum(temp_5[:, 6] == 3)
d = sum(temp_5[:, 6] == 4)
e = sum(temp_5[:, 6] == 5)
Real_reward = sum(temp_5[:, -2])
print(a, b, c, d, e, Real_reward)

In [ ]:
ncf_y0dr = NCF(num_user, num_item, embedding_k = embedding_k)
ncf_y0dr.cuda()
ncf_y0dr.fit(x_tr, ((1-t_tr) * (y_tr/(1-ps_tr) - y0_or/(1-ps_tr)) + y0_or), 
             lamb = 1e-5, lr = 0.01, batch_size = 8192)
y0_dr_te,_ = ncf_y0dr.predict(x_te)

ncf_y1dr = NCF(num_user, num_item, embedding_k = 64)
ncf_y1dr.cuda()
ncf_y1dr.fit(x_tr, (t_tr * (y_tr/(ps_tr) - y1_or/(ps_tr)) + y1_or),
             lamb = 1e-5, lr = 0.01, batch_size = 8192)
y1_dr_te,_ = ncf_y1dr.predict(x_te)

reward_dr = (1 + C) * (y1_dr_te - y0_dr_te) - C
temp_dr = np.c_[constructed_data_test, reward_dr]
temp_dr = temp_dr[np.argsort(-temp_dr[:, -1])]

In [18]:
y1_dr_c1_te = c1_te * y1_dr_te
y1_dr_c0_te = (1-c1_te) * y1_dr_te
y0_dr_c1_te = c1_te * (1-y1_dr_te)
y0_dr_c0_te = (1-c1_te) * (1-y1_dr_te)

pa_dr = y0_dr_c0_te
pb_dr = y1_dr_c0_te
pc_dr = y0_dr_c1_te
pe_dr = y0_dr_te - y1_dr_c0_te
pd_dr = y1_dr_c1_te - pe_dr

if scenario == 'cash':
    cost = np.array([0, 0, -C, 1, -C])
    
if scenario == 'coupon':
    cost = np.array([0, 0, 0, 1, -C])    
    
p_dr = np.c_[pa_dr, pb_dr, pc_dr, pd_dr, pe_dr]    
    
reward_dr = np.sum(p_dr * cost, axis = 1)

temp_dr_ours = np.c_[constructed_data_test, reward_dr]
temp_dr_ours = temp_dr_ours[np.argsort(-temp_dr_ours[:, -1])]

In [ ]:
temp_5 = temp_dr_ours[:int(min(budget, sum(temp_dr_ours[:, -1] > 0)/temp_dr_ours.shape[0]) * temp_dr_ours.shape[0])]
a = sum(temp_5[:, 6] == 1)
b = sum(temp_5[:, 6] == 2)
c = sum(temp_5[:, 6] == 3)
d = sum(temp_5[:, 6] == 4)
e = sum(temp_5[:, 6] == 5)
Real_reward = sum(temp_5[:, -2])
print(a, b, c, d, e, Real_reward)

In [ ]:
temp_5 = temp_dr[:int(min(budget, sum(temp_dr[:, -1] > 0)/temp_dr.shape[0]) * temp_dr.shape[0])]
a = sum(temp_5[:, 6] == 1)
b = sum(temp_5[:, 6] == 2)
c = sum(temp_5[:, 6] == 3)
d = sum(temp_5[:, 6] == 4)
e = sum(temp_5[:, 6] == 5)
Real_reward = sum(temp_5[:, -2])
print(a, b, c, d, e, Real_reward)

In [ ]:
ncf = NCF_ours(num_user, num_item, embedding_k = embedding_k)
ncf.cuda()
ncf.fit(x_tr, t_tr, c_tr, y_tr, eta = 0.5, alpha1 = 5, alpha = 1, beta = 1, theta = 5, gamma = 5, rho = 5, thr = 0.1,
       lr=0.01,
       batch_size=8192,
       lamb=1e-5,
       tol=1e-5,
       verbose=False)

if scenario == 'cash':
    cost = np.array([0, 0, -C, 1, -C])
    
if scenario == 'coupon':
    cost = np.array([0, 0, 0, 1, -C])

pred_class = ncf.predict(constructed_data_test[:, :2]).reshape([-1, 5])
reward_ours = np.sum(pred_class * cost, axis = 1)

temp_ours = np.c_[constructed_data_test, reward_ours]
temp_ours = temp_ours[np.argsort(-temp_ours[:, -1])]

In [ ]:
temp_5 = temp_ours[:int(min(budget, sum(temp_ours[:, -1] > 0)/temp_ours.shape[0]) * temp_ours.shape[0])]
a = sum(temp_5[:, 6] == 1)
b = sum(temp_5[:, 6] == 2)
c = sum(temp_5[:, 6] == 3)
d = sum(temp_5[:, 6] == 4)
e = sum(temp_5[:, 6] == 5)
Real_reward = sum(temp_5[:, -2])
print(a, b, c, d, e, Real_reward)